# Import Section


In [1]:
"""
This script sets up the environment for a Jupyter Notebook that is part of the 
ML_G-sensor project. It helps to prepare the data, training, converting to TFLite and testing the model. 
"""
import shutil
import os
import ipywidgets as widgets
from ipywidgets import interact_manual
from ipywidgets import Layout, Box, Dropdown, Label
from IPython.display import display
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section


In [2]:
class TrainConfigAndCmdsWidgets:
    """
    A class to create and manage widgets for configuring and running training commands for a machine learning model.
    Methods:
    --------
    move_allfiles(src_folder, dst_folder):
        Moves all files from the source folder to the destination folder.
    show_headline(output):
        Displays a headline with the given output text.
    show_main():
        Displays the main interface with tabs for data acquisition, data preparation, training, and testing.
    test_chooser():
        Displays file choosers for selecting the dataset and TFLite file for testing.
    """
    def __init__(self):

        self.tflite_file_loc = ""

        form_item_layout = Layout(
            display="flex",
            flex_flow="row",
            justify_content="space-between",
        )

        # Data acquisition
        self.a_aq = widgets.Text(value="cy", placeholder="Type something", disabled=False)
        self.b_aq = widgets.Text(value="wing", placeholder="Type something", disabled=False)
        self.c_aq = widgets.IntText(value=200, disabled=False)
        self.d_aq = widgets.IntText(value=1, disabled=False)
        self.e_aq = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")
        self.f_aq = widgets.Text(value="COM7", placeholder="Type something", disabled=False)

        form_acquisition_items = [
            Box([Label(value="Person Name"), self.a_aq], layout=form_item_layout),
            Box([Label(value="Action Label"), self.b_aq], layout=form_item_layout),
            Box([Label(value="Collecting Length (points)"), self.c_aq], layout=form_item_layout),
            Box([Label(value="Negative Number (If Action Label is negative)"), self.d_aq], layout=form_item_layout),
            Box([Label(value="COM Port"), self.f_aq], layout=form_item_layout),
            Box([Label(value="Collect Data"), self.e_aq], layout=form_item_layout),
        ]

        self.form_acquisition_cmd = Box(
            form_acquisition_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # train
        self.a_ta = Dropdown(options=["CNN", "CNN-S", "LSTM"])
        self.c_ta = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")
        self.d_ta = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")
        self.e_ta = widgets.IntSlider(value=15, min=10, max=100, step=5)
        self.f_ta = widgets.IntSlider(value=16, min=16, max=256, step=16)

        form_train_items = [
            Box([Label(value="Model Type"), self.a_ta], layout=form_item_layout),
            Box([Label(value="Training Epochs"), self.e_ta], layout=form_item_layout),
            Box([Label(value="Batch Size"), self.f_ta], layout=form_item_layout),
            Box([Label(value="Start Training"), self.c_ta], layout=form_item_layout),
            Box([Label(value="Convert to c"), self.d_ta], layout=form_item_layout),
        ]

        self.form_output_train_cmd = Box(
            form_train_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # data prepare
        self.a_dp = widgets.Textarea(value="ring slope wing", placeholder="Type something", disabled=False)
        self.b_dp = widgets.Textarea(value="cy", placeholder="Type something", disabled=False)
        self.c_dp = widgets.Text(value="out_dataset_1", placeholder="Type something", disabled=False)
        self.d_dp = widgets.IntSlider(value=128, min=32, max=224, step=16)
        self.e_dp = widgets.FloatSlider(value=0.7, min=0.4, max=0.8, step=0.1)
        self.f_dp = widgets.FloatSlider(value=0.1, min=0.05, max=0.4, step=0.05)
        self.g_dp = widgets.IntText(value=30, disabled=False)
        self.h_dp = widgets.IntText(value=50, disabled=False)
        self.i_dp = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_data_prepare_items = [
            Box([Label(value="Label Folder"), self.a_dp], layout=form_item_layout),
            Box([Label(value="Names"), self.b_dp], layout=form_item_layout),
            Box([Label(value="Output Folder"), self.c_dp], layout=form_item_layout),
            Box([Label(value="Sequence Length (points)"), self.d_dp], layout=form_item_layout),
            Box([Label(value="Train Ratio"), self.e_dp], layout=form_item_layout),
            Box([Label(value="Validation Ration"), self.f_dp], layout=form_item_layout),
            Box([Label(value="Random Seed"), self.g_dp], layout=form_item_layout),
            Box([Label(value="Generated Data Number(X3)"), self.h_dp], layout=form_item_layout),
            Box([Label(value="Data Prepare"), self.i_dp], layout=form_item_layout),
        ]

        self.form_data_prepare_cmd = Box(
            form_data_prepare_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # test
        self.b_tt = widgets.Textarea(value="ring slope wing", placeholder="Type something", disabled=False)
        self.c_tt = widgets.IntSlider(value=128, min=32, max=224, step=16)
        self.a_tt = widgets.Button(description="Setting", layout=Layout(width="30%", height="30px"), button_style="success")

        form_test_items = [
            Box([Label(value="Label Folder"), self.b_tt], layout=form_item_layout),
            Box([Label(value="Sequence Length (points)"), self.c_tt], layout=form_item_layout),
            Box([Label(value="Choose the Test Setting"), self.a_tt], layout=form_item_layout),
        ]

        self.form_output_test_cmd = Box(
            form_test_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

    def move_allfiles(self, src_folder, dst_folder):
        """
        Moves all files and directories from the source folder to the destination folder.
        """
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  # copy whole folder
                shutil.move(fullpath, dst_folder)
                print(f"Copy finish: {f}")

    def show_headline(self, output):
        """
        Displays a headline with the specified output text in a styled HTML format.
        Args:
            output (str): The text to be displayed as the headline.
        """
        html0 = widgets.HTML(value=f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)

    def show_main(self):
        """
        Displays the main interface for data preparation and training settings.
        This method creates and displays a tabbed interface with options for data acquisition, data preparation, training, and testing.
        It also sets up interactive widgets and button click handlers for each of these options.
        """

        intro_text = "Please Choose the setting of data prepare & train"
        html_widget = widgets.HTML(value=f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(html_widget)

        # Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[self.form_acquisition_cmd, self.form_data_prepare_cmd, self.form_output_train_cmd, self.form_output_test_cmd]).add_class("parentstyle")
        tab_contents = ["Data Acquisition", "Data Prepare", "Train", "Test"]
        tab.titles = tab_contents

        output_widgets = widgets.Output(layout=Layout(border="1px solid green"))

        def act_para(
            *, model_type, label, person_name, out_fld, seq_len, train_ration, val_ration, rand_seed, data_num, epochs, batch, p_name, action_label, max_len, ne_num, com, label_test, seq_len_test
        ):

            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()

        # ------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        # ------------------#
        out_inter = widgets.interactive_output(
            act_para,
            {
                "model_type": self.a_ta,
                "label": self.a_dp,
                "person_name": self.b_dp,
                "out_fld": self.c_dp,
                "seq_len": self.d_dp,
                "train_ration": self.e_dp,
                "val_ration": self.f_dp,
                "rand_seed": self.g_dp,
                "data_num": self.h_dp,
                "epochs": self.e_ta,
                "batch": self.f_ta,
                "p_name": self.a_aq,
                "action_label": self.b_aq,
                "max_len": self.c_aq,
                "ne_num": self.d_aq,
                "com": self.f_aq,
                "label_test": self.b_tt,
                "seq_len_test": self.c_tt,
            },
        )
        display(tab, out_inter)

        # ------------------#
        # for labelimg cmd, move to outside of act_para to prevent keep trigering
        # ------------------#
        output_widgets = widgets.Output(layout=Layout(border="1px solid green"))
        display(output_widgets)

        def on_button_clicked_data_acq(b):
            with output_widgets:
                clear_output()
                print("Data acquisition")
                self.data_acq()

        self.e_aq.on_click(on_button_clicked_data_acq)

        def on_button_clicked_data_prepare(b):
            with output_widgets:
                clear_output()
                print("Data Prepare")
                self.data_prepare()

        self.i_dp.on_click(on_button_clicked_data_prepare)

        def on_button_clicked_train(b):
            with output_widgets:
                clear_output()
                print("Train. . .")
                self.run_train()

        self.c_ta.on_click(on_button_clicked_train)

        def on_button_clicked_tflu(b):
            with output_widgets:
                clear_output()
                print("Convert to tflu. . .")
                self.convert_tflu()

        self.d_ta.on_click(on_button_clicked_tflu)

        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.test_chooser()

        self.a_tt.on_click(on_button_clicked_test)

    def test_chooser(self):
        """
        Displays file choosers for selecting a dataset and a TFLite model, and provides a button to start testing.
        """
        path_fc = os.path.join(os.getcwd(), "data")
        fc = FileChooser(path_fc)
        fc.show_only_dirs = True
        fc.title = "<b><font color='lightblue'><font size=4>Choose the dataset.</b>"
        display(fc)

        path_ftflite = os.path.join(os.getcwd(), "generated_model")
        f_tflite = FileChooser(path_ftflite)
        f_tflite.filter_pattern = ["*.tflite"]
        f_tflite.title = "<b><font color='lightblue'><font size=4>Choose the Tflite for testing.</b>"
        display(f_tflite)

        def act_test():
            input_dataset = fc.selected_path
            input_dataset = input_dataset.split("\\")[-1]
            input_tflite = f_tflite.selected
            print(f"The chosen dataset: {input_dataset}")
            print(f"The chosen tflite: {input_tflite}")
            self.test_run(input_dataset, input_tflite)
            print("Finish!")

        evt = interact_manual(act_test)
        evt.widget.children[0].description = "Start Testing"  # because there are 3 parameter of the evt
        evt.widget.children[0].button_style = "primary"

    def test_run(self, input_dataset, input_tflite):
        """
        This method runs the `train.py` script with the provided arguments to perform
        training or testing of the model. The parameters are taken from the instance
        attributes and passed to the script.
        """
        %run train.py --folders $self.b_tt.value --out_dir $input_dataset --seq_length $self.c_tt.value \
        --test_tflite True --tflite_path $input_tflite

    def data_acq(self):
        """
        Acquires data using the visual_sparkfun_acc.py script with specified parameters.
        This method runs an external script to collect data from a G-sensor. The parameters for the script
        are provided by the attributes of the class instance.
        """
        %run visual_sparkfun_acc.py --port $self.f_aq.value --user_name $self.a_aq.value --action_label $self.b_aq.value \
        --max_len $self.c_aq.value --negative_num $self.d_aq.value

    def data_prepare(self):
        """
        Prepares the data for the machine learning model by running an external script with specified parameters.
        """
        print(self.a_dp.value)
        %run data_prepare.py --folders $self.a_dp.value --names $self.b_dp.value --out_dir $self.c_dp.value \
        --seq_length $self.d_dp.value --train_ratio $self.e_dp.value --val_ratio $self.f_dp.value \
        --rand_seed $self.g_dp.value --neg_data_num $self.h_dp.value

    def run_train(self):
        """
        Executes the training script with the specified parameters.
        """
        %run train.py --model $self.a_ta.value --folders $self.a_dp.value --out_dir $self.c_dp.value --seq_length $self.d_dp.value \
        --epochs $self.e_ta.value --batch_size $self.f_ta.value
    
    def convert_tflu(self):
        """
        Converts TensorFlow Lite models to TensorFlow Lite Micro models and saves them as C source files.
        This method performs the conversion for both int8 quantized and normal TensorFlow Lite models.
        It generates the corresponding C source files for each model.
        """
        tflite_path = os.path.join("generated_model", f"model_{self.a_ta.value}_int8quantized.tflite")
        output_path = os.path.join("generated_model", f"model_{self.a_ta.value.lower()}_int8quantized.cc")
        print("Int8quantized TFLite:")
        print(f"tflite path: {tflite_path}")
        print(f"C file path: {output_path}")
        %run tflite_to_tflu.py --tflite_path $tflite_path --output_path $output_path

        tflite_path = os.path.join("generated_model", f"model_{self.a_ta.value}.tflite")
        output_path = os.path.join("generated_model", f"model_{self.a_ta.value.lower()}.cc")
        print("Normal TFLite:")
        print(f"tflite path: {tflite_path}")
        print(f"C file path: {output_path}")
        %run tflite_to_tflu.py --tflite_path $tflite_path --output_path $output_path

# Run Section

- The detail description of all the parameters and each step meaning is here [meaning](#id-train_evl_monitor)
- In this notebook step, you have alreay finish the dataset prepared. If not, please go to `image_dataset\create_data.ipynb`.

In [3]:
act = TrainConfigAndCmdsWidgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…